# Converting TensorFlow Model

## Using Frozen Graph

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import array
import cv2

#### Downloading ImageNet test dataset:

In [ ]:
ds, ds_info = tfds.load('imagenet_v2', split='test', as_supervised=True, with_info=True)
ds = tfds.as_numpy(ds)

test_labels = []
img_list = []

#### Functions for conversion and running model:

In [ ]:
NUM_TEST_SAMPLES = 100
IMAGE_SIDE = 128

def preprocess(image):
    image = (image) / 127.5
    return image - 1.0

def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

def quantizationDataGenerator():
    count = 0
    for image, label in ds:
        image = image.copy()
        image = cv2.resize(image, (IMAGE_SIDE, IMAGE_SIDE))
        array = preprocess(image).astype(np.float32)[np.newaxis, ...]
        yield([array])
        count += 1
        if count > 300:
            break

def testDataGenerator():
    count = 0
    for image, label in ds:
        image = image.copy()
        image = cv2.resize(image, (IMAGE_SIDE, IMAGE_SIDE))
#         imshow(image)
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imwrite('img/' + str(label) + '.jpg', img_rgb, [cv2.IMWRITE_JPEG_QUALITY,100])
#         print(img_rgb)
        array = preprocess(image).astype(np.float32)[np.newaxis, ...]
#         print(label)
        test_labels.append(label)
        yield(array)
        count += 1
        if count > 300:
            break
        
def convertTFL(name, model):
    converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph('source_model/mobilenet_v1_0.5_128/mobilenet_v1_0.5_128_frozen.pb',
                                                                    input_arrays = ['input'], output_arrays = ['MobilenetV1/Predictions/Reshape_1'], 
                                                                    input_shapes = {'input' : [1, IMAGE_SIDE, IMAGE_SIDE, 3]})
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = quantizationDataGenerator
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    quantModel = converter.convert()

    with open("{}.tflite".format(name), "wb") as f:
        f.write(quantModel)
        
def runTFL(name):
    from tflite_runtime.interpreter import Interpreter
    interpreter = Interpreter('{}.tflite'.format(name))
    interpreter.allocate_tensors()

    inputTensorIndex = interpreter.get_input_details()[0]['index']
    outputTensorIndex = interpreter.get_output_details()[0]['index']
    input_scale, input_zero_point = interpreter.get_input_details()[0]['quantization']
    output_scale, output_zero_point = interpreter.get_output_details()[0]['quantization']

    
    prediction_values = []
    
    for x in testDataGenerator():
        test_image = np.int8(x / input_scale + input_zero_point)
        test_image = x
        interpreter.set_tensor(inputTensorIndex, test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(outputTensorIndex)
        result = np.argmax(output[0])
        prediction_values.append(result - 1)
    
    accurate_count = 0
    for index in range(len(prediction_values)):
        print(str(prediction_values[index]) + ":" + str(test_labels[index]))
        if prediction_values[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_values)

    return accuracy * 100

#### Convert and run TF Lite model to test it:

In [ ]:
tf.keras.backend.clear_session()

name = 'mobilenet_v1'
convertTFL(name, model)

#### Run model to test it on small dataset. 
Note: accuracy might be low on this specific data.

In [ ]:
runTFL(name)

In order to integrate converted model into TFLM application we have to save it as a C array. One way to do that is to use xxd utility available on Linux or in Cygwin/MinGW terminals on Windows. Open terminal and run following commands:

```
xxd -i mobilenet_v1.tflite > model.h
```
The model is ready to be integrated into TFLM application

## Done